In [ ]:
from numpy import linalg as LA
from copy import deepcopy
import numpy as np
import io
import random

In [ ]:
epochs = 2
alpha = 0.02
u = 138493
m = 27278
ratings_file = "ml-20m/ratings.csv"

# the below 6 files are corresponding to the 3 separate random folds while splitting train and test data
train_ratings_file_1 = "train_data.csv"
test_ratings_file_1 = "test_data.csv"

train_ratings_file_2 = "train_data-2.csv"
test_ratings_file_2 = "test_data-2.csv"

train_ratings_file_3 = "train_data-3.csv"
test_ratings_file_3 = "test_data-3.csv"
movies_mapping_file = "ml-20m/movies.csv"

# These are the values which provided the minimum value of avg. RMSE
lambd = 0.02
rank = 8

In [ ]:
# build movie dictionary with line no as numpy movie id ,its actual movie id as the key.
def build_movies_dict(movies_file):
    i = 0
    movie_id_dict = {}
    with io.open(movies_file, 'r', encoding="utf8") as f:
        for line in f:
            if i == 0:
                i = i+1
            else:
                movieId = line.split(',')[0]
                movie_id_dict[int(movieId)] = i-1
                i = i+1
    return movie_id_dict

In [ ]:
# this function returns 3 dictionaries: train/test_data_dict, user_to_movie_dict and movie_to_user_dict
# the user_to_movie_dict and movie_to_user_dict are used in the optim_calc_error
def read_data_sgd_test(input_file, movies_dict):
    # creating a dictionary, key: (user_id, movie_id), value: rating
    X = {}
    user_to_movie_dict = {}
    movie_to_user_dict = {}

    # because we don't have a header row now
    i = 1

    with open(input_file,'r') as f:
        for line in f:

            # to escape the header row
            if i == 0:
                i += 1
            else:
                user, movie_id, rating, timestamp = line.split(',')
                m_id = movies_dict[int(movie_id)]

                X[(int(user)-1, m_id)] = float(rating)
                
                u_id = int(user)-1
                
                if u_id in user_to_movie_dict:
                    user_to_movie_dict[u_id].append(m_id)
                else:
                    user_to_movie_dict[u_id] = []
                    user_to_movie_dict[u_id].append(m_id)
                
                if m_id in movie_to_user_dict:
                    movie_to_user_dict[m_id].append(u_id)
                else:
                    movie_to_user_dict[m_id] = []
                    movie_to_user_dict[m_id].append(u_id)
                
                i += 1
    return X, user_to_movie_dict, movie_to_user_dict

In [ ]:
movies_dict = build_movies_dict(movies_mapping_file)
train_data_dict, train_user_to_movie_dict, train_movie_to_user_dict = read_data(train_ratings_file, movies_dict)
test_data_dict, test_user_to_movie_dict, test_movie_to_user_dict = read_data(test_ratings_file, movies_dict)

In [ ]:
# this function calculates error overall data points from the scratch
def calc_error(V, W, X):
    error = 0
    V_norm = LA.norm(V)
    W_norm = LA.norm(W)
    for key, value in X.items():
        i = key[0]
        j = key[1]
        rating = value
        error += np.power(rating - np.dot(V[i], W[j]), 2)

    error += (lambd)*(np.power(V_norm, 2) + np.power(W_norm, 2))
    return error

In [ ]:
# this function updates the given error corresponding to the u_id and m_id which have been updated using SGD
def optim_calc_error(V_old, W_old, V_new, W_new, X, u_id, m_id, error):
    
    m_list = train_user_to_movie_dict[u_id]
    u_list = train_movie_to_user_dict[m_id]  
    
    for m in m_list:
        error = error - (np.power(X[u_id, m] - np.dot(V_old[u_id],W_old[m]), 2))
        error = error + np.power(X[u_id, m] - np.dot(V_new[u_id],W_new[m]), 2)
    
    for u in u_list:
        error = error - (np.power(X[u, m_id] - np.dot(V_old[u],W_old[m_id]), 2))
        error = error + np.power(X[u, m_id] - np.dot(V_new[u],W_new[m_id]), 2)
    
    error = error - (lambd/2) * np.sum(np.power(V_old[u_id],2) + pow(W_old[m_id],2))
    error = error + (lambd/2) * np.sum(np.power(V_new[u_id],2) + pow(W_new[m_id],2))
    return error

In [ ]:
# v1 as per the experiment section of write-up
def matrix_factor_sgd_v1(X):
    V = np.random.rand(u, rank)
    W = np.random.rand(m, rank)
    
    # we have to calculate the initial error which the optimal function will then update on
    error = calc_error(V, W, X)
    
    for epoch in range(epochs):
        for key, value in X.items():
            i = key[0]
            j = key[1]
            rating = value
            eij = rating - np.dot(V[i], W[j])
            
            V_old = deepcopy(V)
            W_old = deepcopy(W)

            V_update = V[i] + alpha * (2.0 * eij * W[j] - (lambd*2.0 * V[i]))
            W_update = W[j] + alpha * (2.0 * eij * V[i] - (lambd*2.0 * W[j]))

            V[i] = V_update
            W[j] = W_update
            
            print(optim_calc_error(V_old, W_old, V, W, X, i, j, error))
            

    return V, W

In [ ]:
# v2 as per the experiment section of write-up
def matrix_factor_sgd_v2(X):
    V = np.random.rand(u, rank)
    W = np.random.rand(m, rank)
    
    for epoch in range(46):
        for key, value in random.sample(X.items(), 1000):
            i = key[0]
            j = key[1]
            rating = value
            eij = rating - np.dot(V[i], W[j])

            V_update = V[i] + alpha * (2.0 * eij * W[j] - (lambd*2.0 * V[i]))
            W_update = W[j] + alpha * (2.0 * eij * V[i] - (lambd*2.0 * W[j]))

            V[i] = V_update
            W[j] = W_update
            
            print(calc_error(V, W, X))

    return V, W

In [ ]:
matrix_factor_sgd_v1(train_data_dict)

In [ ]:
matrix_factor_sgd_v2(train_data_dict)

Based on the print statements of the above two function calls, I used that data to create learning curves which are provided in the pdf write-up.